# Equations and theoretical model

#### Theoretical model
<div>
<img src="img/pendulum_coords.png" style="margin-right:10px"//>
<img src="img/polar_coords.png"/>
</div>
<br>

The cartesian coordinates o-xyz represent the pendulum's coordinate system. In order to describe its position we use the angles $\alpha$ and $\beta$:
> x = L $\sin$$\alpha$ $\cos$$\beta$

> y = L $sin$$\alpha$ $\sin$$\beta$

> z = $h_{0}$ - L $\cos$$\alpha$


Where: L represents the pendulum's length
<br>

Besides the pendulum's oscillating motion, it also rotates with the earth. In order to describe the angular velocity we require an assistant coordinate system O-XYZ. Here, the pendulum sits on a fixed point described using the spherical coordinates $\phi$ (latitude) and $\theta$ (longitude).